# File Name: simple_guardrail_creation.ipynb
### Location: Chapter 8
### Purpose: 
#####             1. Set up an Amazon Bedrock guardrail using the API.
#####             2. Test and monitor the guardrail during prompt interactions.
#####             3. Test and monitor the guardrail during response handling.
#####             4. Execute and monitor the complete guardrail policy.
##### Dependency: simple-sageMaker-bedrock.ipynb at Chapter 3 should work properly. 
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84"

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import botocore
import boto3
import warnings
import time
import random

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

## Define important environment variable

In [ ]:
%%time

# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    boto3_session_name = boto3.session.Session()

    # Retrieve the current AWS region from the session
    aws_region_name = boto3_session_name.region_name
    
    # Create a new Boto3 bedrock client to interact with AWS services
    boto3_bedrock_client = boto3.client('bedrock')
    
    # Create a new Boto3 bedrock runtime client to interact with AWS services
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime')
    
    # Create a new Boto3 bedrock runtime client to interact with AWS services
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime')
    
    # Generate a random suffix number between 200 and 900
    random_suffix = random.randrange(200, 900)
    
    # Store all variables in a dictionary
    variables_store = {
        "boto3_session_name": boto3_session_name,
        "aws_region_name": aws_region_name,
        "boto3_bedrock_client": boto3_bedrock_client,
        "random_suffix": random_suffix,
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client
    }

    # Print all variables
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


# Set up an Amazon Bedrock guardrail using the API


###### Guardrail Creation:

###### Calls the create_guardrail method on the client instance.
###### Passes various configurations as parameters:
###### Name and Description: Sets the name to "misleading-financial-guidance" and provides a description of the type of advice the guardrail aims to address.
###### Topic Policy Configuration: Specifies the topics related to misleading financial guidance, along with definitions and examples of such guidance, and sets the type to "DENY."
###### Content Policy Configuration: Defines filters for various types of inappropriate content (e.g., sexual, violence, hate) with specified strengths.
###### Word Policy Configuration: Includes a list of words related to financial services and a managed word list for profanity.
###### Sensitive Information Policy Configuration: Configures actions for different types of personally identifiable information (PII), such as anonymizing emails and phone numbers, and blocking social security and bank account numbers. It also includes a regex pattern for detecting account numbers.
###### Contextual Grounding Policy Configuration: Sets thresholds for grounding and relevance filters.
###### Blocked Input Messaging: Provides a message to users when their input is blocked, directing them to customer service and advising against sharing sensitive information.
###### Blocked Output Messaging: Similar to input messaging but for output responses, guiding users on how to ask general product questions.
###### Tags: Adds metadata tags indicating the purpose and environment (production) of the guardrail.

In [ ]:
%%time

def create_guardrail(client, guardrail_name ):
    try:
        create_response = client.create_guardrail(
            name=guardrail_name,
            description='Advice that misleads or misinforms on financial matters, often leading to poor decision-making and potential financial loss. Typically based on incomplete, biased, or incorrect information.',
            topicPolicyConfig={
                'topicsConfig': [
                    {
                        'name': 'Misleading financial guidance',
                        'definition': 'Offering personalized guidance or recommendations on managing financial assets, investments, or trusts in a financial advisor role, with associated obligations and liabilities.',
                        'examples': [
                            'Which stocks are best suited for short-term and long-term investments?',
                            'Is investing in a mutual fund or bitcoins a wise choice in the current market trend?',
                            'How should I allocate my 401(k) for optimal growth?',
                            'What type of trust fund would be ideal for securing my family long-term?',
                            'Would hiring a financial advisor from XXX organization improve the management of my investments?'
                        ],
                        'type': 'DENY'
                    }
                ]
            },
            contentPolicyConfig={
                'filtersConfig': [
                    {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'HATE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'INSULTS', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'MISCONDUCT', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'PROMPT_ATTACK', 'inputStrength': 'HIGH', 'outputStrength': 'NONE'}
                ]
            },
            wordPolicyConfig={
                'wordsConfig': [
                    {'text': 'fiduciary insights'},
                    {'text': 'investment guidance'},
                    {'text': 'stock advice'},
                    {'text': 'financial planning'},
                    {'text': 'portfolio strategy'},
                    {'text': 'retirement planning'},
                    {'text': 'wealth insights'},
                    {'text': 'trust planning'},
                    {'text': 'investment strategy'},
                    {'text': 'advisor insights'}
                ],
                'managedWordListsConfig': [
                    {'type': 'PROFANITY'}
                ]
            },
            sensitiveInformationPolicyConfig={
                'piiEntitiesConfig': [
                    {'type': 'EMAIL', 'action': 'ANONYMIZE'},
                    {'type': 'PHONE', 'action': 'ANONYMIZE'},
                    {'type': 'NAME', 'action': 'ANONYMIZE'},
                    {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
                    {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
                    {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
                ],
                'regexesConfig': [
                    {
                        'name': 'Account Number',
                        'description': 'Detects account numbers with six initial digits followed by four trailing digits (e.g., 1234567890)',
                        'pattern': r'\b\d{6}\d{4}\b',
                        'action': 'ANONYMIZE'
                    }
                ]
            },
            contextualGroundingPolicyConfig={
                'filtersConfig': [
                    {'type': 'GROUNDING', 'threshold': 0.8},
                    {'type': 'RELEVANCE', 'threshold': 0.8}
                ]
            },
            blockedInputMessaging="""I can provide general information about Anycompany Financial Services’ products and offerings. However, for personalized assistance or specific inquiries, please reach out directly to our customer service team. For your security, please avoid sharing any sensitive information through this channel. Feel free to ask general product questions here without including personal details.""",
            blockedOutputsMessaging="""While I'm able to share general information about Anycompany Financial Services’ products and services, I can’t fully address your request here. For personalized assistance or specific questions, please reach out to our customer service team directly. For your security, please avoid sharing any sensitive information on this platform. If you have general questions about our products, feel free to ask, but please exclude any personal details.""",
            tags=[
                {'key': 'purpose', 'value': 'misleading-financial-guidance'},
                {'key': 'environment', 'value': 'production'}
            ]
        )

        print(create_response)
        
        return create_response

    except Exception as e:
        print(f"An error occurred while creating the guardrail: {str(e)}")

# Usage example
# client = your_client_instance  # Replace with your actual client instance

name='misleading-financial-guidance'
guardrail_name = name + "-" + str(random_suffix)
gr_create_response = create_guardrail(boto3_bedrock_client, guardrail_name )

###### Note: 
###### You can check the Amazon Bedrock Guardrail from the console also after the creation. 
###### Open Amazon Bedrock from console. Click <ins>SafeGuards</ins> from right side navigation panel. Click <ins>Guardrails</ins>. 

# Getting a guardrail, generating its versions, and enumerating all versions and drafts.

###### <ins>get_guardrail(client, guardrail_id):</ins>
###### Retrieves the draft version of a specified guardrail using its identifier.
###### If successful, it returns the guardrail's response; if an error occurs, it prints an error message and returns None.

###### <ins>create_guardrail_version(client, guardrail_id, description):</ins>
###### Creates a new version of the specified guardrail with a given description.
###### It returns the response upon success or prints an error message and returns None in case of failure.

###### <ins>list_guardrails(client, guardrail_arn, max_results=5):</ins>
###### Lists all guardrails associated with the specified ARN, with an optional parameter to limit the number of results.
###### Similar to the previous functions, it returns the response if successful or handles errors appropriately.

In [ ]:
%%time

def get_guardrail(client, guardrail_id):
    try:
        response = client.get_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion='DRAFT'
        )
        return response
    except Exception as e:
        print(f"Error getting guardrail: {e}")
        return None


def create_guardrail_version(client, guardrail_id, description):
    try:
        response = client.create_guardrail_version(
            guardrailIdentifier=guardrail_id,
            description=description
        )
        return response
    except Exception as e:
        print(f"Error creating guardrail version: {e}")
        return None


def list_guardrails(client, guardrail_arn, max_results=5):
    try:
        response = client.list_guardrails(
            guardrailIdentifier=guardrail_arn,
            maxResults=max_results
        )
        return response
    except Exception as e:
        print(f"Error listing guardrails: {e}")
        return None


# Main section execution
guardrail_id = gr_create_response['guardrailId']
guardrail_arn = gr_create_response['guardrailArn']
description = 'Version of Guardrail'

# Get guardrail
draft_guardrail = get_guardrail(boto3_bedrock_client, guardrail_id)
if draft_guardrail:
    print("Draft Guardrail:", draft_guardrail)
    print()
    print()

# Create a version for the guardrail
version_response = create_guardrail_version(boto3_bedrock_client, guardrail_id, description)
if version_response:
    print("Guardrail Version Created:", version_response)
    print()
    print()

# List all versions of guardrails
list_guardrails_response = list_guardrails(boto3_bedrock_client, guardrail_arn)
if list_guardrails_response:
    print("List of Guardrails:", list_guardrails_response)
    print()
    print()

# Modifying a Guardrail

###### Modify topicPolicyConfig examples. 
###### Old example: Is investing in a mutual fund and bitcoins a wise choice in the current market trend?
###### New example: Is investing in a mutual fund,bitcoins, real estate a wise choice in the current market trend?


###### <ins>Guardrail Update</ins>:
###### Updates an existing guardrail with a unique identifier, description, and topic-specific policies for handling misleading financial guidance. Topics include managing financial assets, investment advice, and trust fund planning, with an action type set to "DENY" for such recommendations.
    
###### <ins>Content Policy</ins>:
###### Specifies filters with a high-strength setting for sensitive content categories like sexual, violent, hate, and insulting language. It also includes a "PROMPT_ATTACK" filter to reduce potentially harmful prompt inputs.
    
###### <ins>Word and Managed Word List Policies</ins>:
###### Defines allowable words related to financial advice (e.g., "fiduciary insights," "investment guidance") while blocking profanity.

###### <ins>Sensitive Information Policy</ins>:
###### Anonymizes specific PII (e.g., emails, phone numbers, names) and blocks U.S. social security, bank account, and credit card numbers. An additional regex detects and anonymizes account numbers.

###### <ins>Contextual Grounding Policy</ins>:
###### Configures thresholds for content grounding and relevance to ensure accurate information.

###### <ins>Blocked Message Responses</ins>:
###### Customizes responses for blocked inputs and outputs, advising users to contact customer service for specific queries and to avoid sharing sensitive information.

###### <ins>Error Handling</ins>:
###### Captures and prints errors if guardrail creation fails.

In [ ]:
%%time

def update_guardrail(client, guardrail_name):
    try:
        create_response = client.update_guardrail(
            guardrailIdentifier=guardrail_arn,
            name=guardrail_name,
            description='Advice that misleads or misinforms on financial matters, often leading to poor decision-making and potential financial loss. Typically based on incomplete, biased, or incorrect information.',
            topicPolicyConfig={
                'topicsConfig': [
                    {
                        'name': 'Misleading financial guidance',
                        'definition': 'Offering personalized guidance or recommendations on managing financial assets, investments, or trusts in a financial advisor role, with associated obligations and liabilities.',
                        'examples': [
                            'Which stocks are best suited for short-term and long-term investments?',
                            'Is investing in a mutual fund,bitcoins, real estate a wise choice in the current market trend?',
                            'How should I allocate my 401(k) for optimal growth?',
                            'What type of trust fund would be ideal for securing my family long-term?',
                            'Would hiring a financial advisor from XXX organization improve the management of my investments?'
                        ],
                        'type': 'DENY'
                    }
                ]
            },
            contentPolicyConfig={
                'filtersConfig': [
                    {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'HATE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'INSULTS', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'MISCONDUCT', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                    {'type': 'PROMPT_ATTACK', 'inputStrength': 'HIGH', 'outputStrength': 'NONE'}
                ]
            },
            wordPolicyConfig={
                'wordsConfig': [
                    {'text': 'fiduciary insights'},
                    {'text': 'investment guidance'},
                    {'text': 'stock advice'},
                    {'text': 'financial planning'},
                    {'text': 'portfolio strategy'},
                    {'text': 'retirement planning'},
                    {'text': 'wealth insights'},
                    {'text': 'trust planning'},
                    {'text': 'investment strategy'},
                    {'text': 'advisor insights'}
                ],
                'managedWordListsConfig': [
                    {'type': 'PROFANITY'}
                ]
            },
            sensitiveInformationPolicyConfig={
                'piiEntitiesConfig': [
                    {'type': 'EMAIL', 'action': 'ANONYMIZE'},
                    {'type': 'PHONE', 'action': 'ANONYMIZE'},
                    {'type': 'NAME', 'action': 'ANONYMIZE'},
                    {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
                    {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
                    {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
                ],
                'regexesConfig': [
                    {
                        'name': 'Account Number',
                        'description': 'Detects account numbers with six initial digits followed by four trailing digits (e.g., 1234567890)',
                        'pattern': r'\b\d{6}\d{4}\b',
                        'action': 'ANONYMIZE'
                    }
                ]
            },
            contextualGroundingPolicyConfig={
                'filtersConfig': [
                    {'type': 'GROUNDING', 'threshold': 0.8},
                    {'type': 'RELEVANCE', 'threshold': 0.8}
                ]
            },
            blockedInputMessaging="""I can provide general information about Anycompany Financial Services’ products and offerings. However, for personalized assistance or specific inquiries, please reach out directly to our customer service team. For your security, please avoid sharing any sensitive information through this channel. Feel free to ask general product questions here without including personal details.""",
            blockedOutputsMessaging="""While I'm able to share general information about Anycompany Financial Services’ products and services, I can’t fully address your request here. For personalized assistance or specific questions, please reach out to our customer service team directly. For your security, please avoid sharing any sensitive information on this platform. If you have general questions about our products, feel free to ask, but please exclude any personal details.""",
        )

        print(create_response)
        
        return create_response

    except Exception as e:
        print(f"An error occurred while creating the guardrail: {str(e)}")

# Usage example
# client = your_client_instance  # Replace with your actual client instance
gr_modify_response = update_guardrail(boto3_bedrock_client,guardrail_name)

# Getting a guardrail, generating its versions, and enumerating all versions and drafts.

In [ ]:
%%time

# Main section execution
guardrail_id = gr_modify_response['guardrailId']
guardrail_arn = gr_modify_response['guardrailArn']
description = 'Version of Guardrail'

# Get guardrail
draft_guardrail = get_guardrail(boto3_bedrock_client, guardrail_id)
if draft_guardrail:
    print("Draft Guardrail:", draft_guardrail)
    print()
    print()

# Create a version for the guardrail
version_response = create_guardrail_version(boto3_bedrock_client, guardrail_id, description)
if version_response:
    print("Guardrail Version Created:", version_response)
    print()
    print()

guardrail_version = version_response['version']

# List all versions of guardrails
list_guardrails_response = list_guardrails(boto3_bedrock_client, guardrail_arn)
if list_guardrails_response:
    print("List of Guardrails:", list_guardrails_response)
    print()
    print()

In [ ]:
%%time

def apply_guardrail(client, guardrail_id, guardrail_version, content, source='INPUT'):
    """
    Applies a guardrail to the given content and returns the response.
    
    Parameters:
        client: The boto3 client instance for Bedrock runtime.
        guardrail_id: Identifier for the guardrail.
        guardrail_version: Version of the guardrail to apply.
        content: List of content items to be analyzed by the guardrail.
        source: Specifies whether the source is 'INPUT' or 'OUTPUT'.
    
    Returns:
        Response from the ApplyGuardrail API if successful, or None if an error occurs.
    """
    try:
        response = client.apply_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion=guardrail_version,
            source=source,
            content=content
        )
        return response
    except Exception as e:
        print(f"Error applying guardrail: {e}")
        return None


def process_guardrail_response(response):
    """
    Processes the response from the ApplyGuardrail API.
    
    Parameters:
        response: Response object from the ApplyGuardrail API.
    """
    if not response:
        print("No response to process.")
        return

    print("API Response:")
    print(json.dumps(response, indent=2))

    # Check the action taken by the guardrail
    if response.get('action') == 'GUARDRAIL_INTERVENED':
        print("\nGuardrail intervened. Output:")
        for output in response.get('outputs', []):
            print(output.get('text', 'No output text available'))
    else:
        print("\nGuardrail did not intervene.")

# Testing the Guardrail


###### <ins>apply_guardrail(client, guardrail_id, guardrail_version, content, source='INPUT')</ins>:
###### This function takes in a Boto3 client instance for Bedrock runtime, a guardrail identifier, a version of the guardrail, the content to be analyzed, and an optional source parameter (defaulting to 'INPUT').
###### It attempts to apply the specified guardrail to the given content by invoking the apply_guardrail API.
###### If the application is successful, it returns the response from the API. If an error occurs, it catches the exception, prints an error message, and returns None.

###### <ins>process_guardrail_response(response)</ins>:
###### This function processes the response returned from the apply_guardrail API.
###### It first checks if the response is None, indicating no data to process, and prints an appropriate message if so.
###### If there is a response, it checks whether the guardrail intervened by examining the 'action' field. If the action indicates that the guardrail intervened, it prints the outputs provided in the response. If the guardrail did not intervene, it prints a corresponding message.

In [ ]:
%%time

def apply_guardrail(client, guardrail_id, guardrail_version, content, source='INPUT'):
    """
    Applies a guardrail to the given content and returns the response.
    
    Parameters:
        client: The boto3 client instance for Bedrock runtime.
        guardrail_id: Identifier for the guardrail.
        guardrail_version: Version of the guardrail to apply.
        content: List of content items to be analyzed by the guardrail.
        source: Specifies whether the source is 'INPUT' or 'OUTPUT'.
    
    Returns:
        Response from the ApplyGuardrail API if successful, or None if an error occurs.
    """
    try:
        response = client.apply_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion=guardrail_version,
            source=source,
            content=content
        )
        return response
    except Exception as e:
        print(f"Error applying guardrail: {e}")
        return None


def process_guardrail_response(response):
    """
    Processes the response from the ApplyGuardrail API.
    
    Parameters:
        response: Response object from the ApplyGuardrail API.
    """
    if not response:
        print("No response to process.")
        return

    # Check the action taken by the guardrail
    if response.get('action') == 'GUARDRAIL_INTERVENED':
        print("\nGuardrail intervened. Output:")
        for output in response.get('outputs', []):
            print(output.get('text', 'No output text available'))
    else:
        print("\nGuardrail did not intervene.")

# Test and monitor the guardrail during prompt interactions.

In [ ]:
%%time

# Define content to be analyzed
content = [
        {
            "text": {
                "text": "Provide investment guidance. My email address is xxxxxx@xxx.com"
            }
        }
    ]
    
    
# Apply guardrail and process response
response = apply_guardrail(boto3_bedrock_runtime_client, guardrail_id, guardrail_version, content, source='INPUT')

### Print response for detail monitoring purpose

In [ ]:
print("API Response:")
print(json.dumps(response, indent=2))
print()

### Print response for guardrail output

In [ ]:
process_guardrail_response(response)

# Test and monitor the guardrail during response handling.

In [ ]:
%%time

# Define content to be analyzed
content = [
    {
        "text": {
            "text": "How should I invest for long term? I want to be able to generate $10,000 a month",
            "qualifiers": ["grounding_source"],
        }
    },
    {
        "text": {
            "text": "What is the Guaranteed return rate of bitcoins?",
            "qualifiers": ["query"],
        }
    },
    {
        "text": {
            "text": "Guaranteed Rate is 7% for real estate investment at USA.",
            "qualifiers": ["guard_content"],
        }
    },
]
    
# Apply guardrail and process response
response = apply_guardrail(boto3_bedrock_runtime_client, guardrail_id, guardrail_version, content, source='OUTPUT')

### Print response for detail monitoring purpose

In [ ]:
print("API Response:")
print(json.dumps(response, indent=2))
print()

### Print response for guardrail output

In [ ]:
process_guardrail_response(response)

# Execute and monitor the complete guardrail policy.


###### <ins>create_payload(model_id, anthropic_version, messages)</ins>:
###### This function constructs a dictionary representing the payload required to make a request to the Bedrock model. It includes: The model ID.
###### Content type and acceptance headers, both set to application/json.
###### A body containing the anthropic version, a maximum token limit, and user messages formatted for the model.

###### <ins>invoke_model(client, payload, guardrail_id)</ins>:
###### This function takes a Boto3 client, the payload created by the create_payload function, and a guardrail ID as inputs. It performs the following:
###### Converts the payload body to bytes.
###### Invokes the model using the provided client with specified parameters, including the guardrail ID and version.
###### Implements error handling to catch exceptions during the model invocation, printing an error message if an exception occurs. It returns the model's response or None if an error occurs.

###### <ins>process_response(response)</ins>:
###### This function processes and prints the response from the model invocation. It checks if the response is valid:
###### If valid, it reads the response body, decodes it, and prints it in a formatted JSON structure.
###### If no response is received, it prints a message indicating that no response was obtained.

In [ ]:
%%time

def create_payload(model_id, anthropic_version, messages):
    """Create the payload for the Bedrock request."""
    return {
        "modelId": model_id,
        "contentType": "application/json",
        "accept": "application/json",
        "body": {
            "anthropic_version": anthropic_version,
            "max_tokens": 1000,
            "messages": [
                {
                    "role": "user",
                    "content": messages
                }
            ]
        }
    }

def invoke_model(client, payload, guardrail_id,guardrail_version):
    """Invoke the model with the specified payload and guardrail."""
    try:
        # Convert the payload body to bytes
        body_bytes = json.dumps(payload['body']).encode('utf-8')

        # Invoke the model
        response = client.invoke_model(
            body=body_bytes,
            contentType=payload['contentType'],
            accept=payload['accept'],
            modelId=payload['modelId'],
            guardrailIdentifier=guardrail_id,
            guardrailVersion=guardrail_version,
            trace="ENABLED"
        )
        return response
    except Exception as e:
        print(f"An error occurred while invoking the model: {str(e)}")
        return None

def process_response(response):
    """Process and print the response from the model."""
    if response is not None:
        response_body = response['body'].read().decode('utf-8')
        print(json.dumps(json.loads(response_body), indent=2))
    else:
        print("No response received.")

In [ ]:
%%time

# Main section execution
bedrock_model_id = "anthropic.claude-3-haiku-20240307-v1:0"
anthropic_version = "bedrock-2023-05-31"

messages = [
        {
            "type": "text",
            "text": "How should I invest for long term? I want to be able to generate $10,000 a month"
        }
    ]

# Create the payload
payload = create_payload(bedrock_model_id, anthropic_version, messages)

# Invoke the model
response = invoke_model(boto3_bedrock_runtime_client, payload, guardrail_id,guardrail_version)

# Process and print the response
process_response(response)

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html


#### <ins>Step 2</ins> 

#### If you are not executing any further lab of this Chapter 8
##### Uncomment and execute the below code to delete the guardrails

In [ ]:
'''
try:
    # Attempt to delete the guardrail using the specified identifier
    response = boto3_bedrock_client.delete_guardrail(
        guardrailIdentifier=guardrail_id
    )
    # Log or print a success message if the deletion is successful
    print(f"Guardrail with ID '{guardrail_id}' deleted successfully.")
    # Optionally, handle the response object if needed
    print(response)

except ClientError as e:
    # Handle client-side errors (e.g., invalid parameters, resource not found)
    print(f"ClientError occurred: {e.response['Error']['Message']}")

except BotoCoreError as e:
    # Handle broader BotoCore errors (e.g., network issues, general SDK errors)
    print(f"BotoCoreError occurred: {str(e)}")

except Exception as e:
    # Handle any unexpected errors not covered by specific exceptions
    print(f"An unexpected error occurred: {str(e)}")
'''